Clean up

In [1]:
from datetime import datetime, timedelta
import time
import pandas as pd
import os

In [2]:
def cleanUp(df):
  #nullwaarden verwijderen
  df.dropna(inplace=True)

  #tijd aanpassen
  #tijdformaat ..:..:..
  tijd_regex = r'^\d{2}:\d{2}:\d{2}$'
  #omzetten naar datetime
  df['date'] = pd.to_datetime(df['dateDiff']).dt.date
  #filter rijen met formaat
  df = df[df['startTime'].str.match(tijd_regex, na=False) & df['rLength'].str.match(tijd_regex, na=False)].copy()
  #afleveringlengte naar seconden omzetten 
  df['Lengte_sec'] = pd.to_timedelta(df['rLength']).dt.total_seconds().astype(int)
  #kijkers naar int
  df['Kijkers'] = df['rateInK'].dropna().astype(str).str.replace('.', '', regex=False).astype(int)

  #uren met 24+
  def time_cor(rij):
    tijdArr = rij['startTime'].split(':')
    if int(tijdArr[0]) >= 24:
      tijdArr[0] = str(int(tijdArr[0]) - 24).zfill(2)
      rij['date'] += timedelta(days=1)
    rij['startTime'] = ':'.join(tijdArr)
    return rij
  
  df = df.apply(time_cor, axis=1)

  #1 kolom voor beide data
  df['FullDate'] = pd.to_datetime(df['date'].astype(str) 
                                  + " " + df['startTime'].astype(str))
  
  #hour en minute voor join later on
  df['hour'] = pd.to_datetime(df['startTime']).dt.hour
  df['minute'] = pd.to_datetime(df['startTime']).dt.minute

  #kolommen verwijderen die niet nodig meer zijn
  df.drop(['startTime', 'rLength', 'rateInK', 'ranking', 'live'], axis=1, inplace=True)

  #de nieuwe dataframe
  df = df[['FullDate', 'date', 'hour','minute', 'channel', 'description', 'Lengte_sec', 'Kijkers']]

  #hernoemen kolommen
  df.rename(columns={'description':'Programma', 'channel':'Kanaal'}, inplace=True)

  return df

In [3]:
Kijkcijfers = pd.read_csv('./data csv/kijkcijfersdataraw.csv')
Kijkcijfers = cleanUp(Kijkcijfers)
Kijkcijfers.sample(10)

C:\Users\krist\AppData\Local\Temp\ipykernel_31964\78328727.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['startTime']).dt.hour
C:\Users\krist\AppData\Local\Temp\ipykernel_31964\78328727.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['minute'] = pd.to_datetime(df['startTime']).dt.minute


,FullDate,date,hour,minute,Kanaal,Programma,Lengte_sec,Kijkers
57222,2024-07-16 21:39:53,2024-07-16,21,39,VRT 1,VIVE LE VELO,4159,562728
11356,2018-04-23 18:59:53,2018-04-23,18,59,VTM,NIEUWS 19U VTM,3089,561036
46931,2023-02-17 22:33:10,2023-02-17,22,33,VTM,9-1-1,2475,194047
2597,2017-02-08 21:35:07,2017-02-08,21,35,VIJF,TEMPTATION ISLAND,2678,397482
61235,2025-02-01 20:36:30,2025-02-01,20,36,VTM,THE HITMAN'S BODYGUARD,6323,221246
12582,2018-06-23 21:25:01,2018-06-23,21,25,CANVAS,SCOTT & BAILEY,2766,193838
15685,2018-11-24 20:03:49,2018-11-24,20,3,CANVAS,VRANCKX,1442,166869
53636,2024-01-18 20:16:49,2024-01-18,20,16,PLAY4,HUIZENJAGERS,2631,246306
60745,2025-01-08 18:28:46,2025-01-08,18,28,VRT 1,BLOKKEN,1697,629349
5477,2017-07-03 13:36:46,2017-07-03,13,36,EEN,WIELRENNEN. RONDE VAN FRANKRIJK,14782,396277


Weerdata mergen

In [4]:
weerData = pd.read_csv('./data csv/weerdataraw.csv')
weerData['timestamp'] = pd.to_datetime(weerData['timestamp'])
#naar zelfde formaat als kijkcijfer datum
weerData['datetime'] = weerData['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

#hour voor join later on
weerData['hour'] = pd.to_datetime(weerData['datetime']).dt.hour
weerData['minute'] = pd.to_datetime(weerData['datetime']).dt.minute
weerData['date'] = pd.to_datetime(weerData['datetime']).dt.date

#verwijder kolom
weerData = weerData.drop(columns=['timestamp'])

weerData = weerData[['datetime', 'date' ,'hour', 'minute', 'temperature_2m', 'apparent_temperature', 
                           'rain', 'snowfall', 'weather_code', 'cloud_cover', 
                           'wind_speed_10m', 'sunshine_duration']]

#hernoemen kolommen
weerData.rename(columns={'temperature_2m':'Temperatuur', 'apparent_temperature':'Gevoelstemp', 'wind_speed_10m': 'Windsnelheid', 'rain':'Regen', 'snowfall': 'Sneeuw', 'weather_code':'Weercode', 'cloud_cover':'Bewolking', 'sunshine_duration':'Zonnenschijn'}, inplace=True)

weerData.sample(10)

,datetime,date,hour,minute,Temperatuur,Gevoelstemp,Regen,Sneeuw,Weercode,Bewolking,Windsnelheid,Zonnenschijn
55383,2023-01-25 15:00:00,2023-01-25,15,0,-0.4,-3.9,0.0,0.0,3,100,6.6,1883.6
12454,2018-03-03 22:00:00,2018-03-03,22,0,1.5,-2.2,0.0,0.0,3,100,10.6,0.0
31481,2020-05-04 17:00:00,2020-05-04,17,0,15.0,11.6,0.0,0.0,2,51,16.0,458.2
47452,2022-03-01 04:00:00,2022-03-01,4,0,3.8,-0.0,0.0,0.0,3,100,10.8,0.0
69935,2024-09-22 23:00:00,2024-09-22,23,0,17.3,18.5,0.0,0.0,2,78,3.9,0.0
73689,2025-02-26 09:00:00,2025-02-26,9,0,2.7,-0.5,0.0,0.0,0,0,10.3,0.0
17891,2018-10-16 11:00:00,2018-10-16,11,0,18.0,17.0,0.0,0.0,1,20,7.8,3600.0
50035,2022-06-16 19:00:00,2022-06-16,19,0,24.1,23.0,0.0,0.0,2,53,8.3,3600.0
3688,2017-03-03 16:00:00,2017-03-03,16,0,10.1,7.1,0.0,0.0,3,100,13.3,0.0
31185,2020-04-22 09:00:00,2020-04-22,9,0,12.3,7.7,0.0,0.0,0,0,19.6,3600.0


Mergen weerdata en kijkcijferdata op FullDate

In [5]:
kijkcijfersWeer = pd.merge(Kijkcijfers, weerData, on=['date', 'hour'], how='left')

kijkcijfersWeer = kijkcijfersWeer[['FullDate', 'date', 'hour', 'Kanaal', 'Programma', 'Lengte_sec', 'Kijkers', 'Temperatuur', 'Gevoelstemp', 'Regen', 'Sneeuw', 'Weercode', 'Bewolking', 'Windsnelheid', 'Zonnenschijn']]

kijkcijfersWeer.sample(10)

#kijkcijfersWeer.to_csv('./data csv/blablableble.csv')

,FullDate,date,hour,Kanaal,Programma,Lengte_sec,Kijkers,Temperatuur,Gevoelstemp,Regen,Sneeuw,Weercode,Bewolking,Windsnelheid,Zonnenschijn
46713,2023-03-21 20:03:23,2023-03-21,20,VTM,FAMILIE,1489,516640,11.1,8.3,0.0,0.0,0,19,16.0,1049.98
21514,2019-09-18 21:43:28,2019-09-18,21,Q2,RAMBO IV,4616,251466,13.2,10.4,0.0,0.0,0,7,13.2,0.00
34981,2021-08-08 13:02:40,2021-08-08,13,EEN,HET 1 UUR-JOURNAAL,1194,630356,18.9,16.3,0.0,0.0,1,22,26.1,3600.00
25699,2020-04-15 20:39:54,2020-04-15,20,EEN,THUIS,1497,1318215,13.9,11.3,0.0,0.0,0,0,8.4,3600.00
46979,2023-04-03 18:12:05,2023-04-03,18,EEN,DAGELIJKSE KOST,929,274039,9.2,4.2,0.0,0.0,0,0,17.6,3600.00
28846,2020-10-04 18:11:54,2020-10-04,18,EEN,DAGELIJKSE KOST,2722,430129,13.6,9.0,0.0,0.0,2,75,25.4,3600.00
717,2016-11-07 23:00:52,2016-11-07,23,VTM,TELEFACTS,3386,203478,3.2,0.8,0.0,0.0,2,70,4.9,0.00
13585,2018-08-16 21:37:00,2018-08-16,21,VTM,MET VIER IN BED,2033,384410,19.1,19.5,0.7,0.0,53,50,13.0,435.10
56193,2024-07-20 18:59:49,2024-07-20,18,VTM,NIEUWS 19U VTM,2717,386360,27.3,29.6,0.0,0.0,2,77,6.6,3600.00
41641,2022-07-05 20:29:46,2022-07-05,20,VTM,GROETEN UIT...,3290,208142,17.8,15.2,0.0,0.0,1,47,14.3,3600.00


debug box

In [6]:
# Bekijk de kolomnamen in Kijkcijfers en weerData
print(Kijkcijfers.columns)
print(weerData.columns)

#nullrows aanwezig?
print(f"null-rows: \n{kijkcijfersWeer.isnull().sum()}")



Index(['FullDate', 'date', 'hour', 'minute', 'Kanaal', 'Programma',
       'Lengte_sec', 'Kijkers'],
      dtype='object')
Index(['datetime', 'date', 'hour', 'minute', 'Temperatuur', 'Gevoelstemp',
       'Regen', 'Sneeuw', 'Weercode', 'Bewolking', 'Windsnelheid',
       'Zonnenschijn'],
      dtype='object')
null-rows: 
FullDate        0
date            0
hour            0
Kanaal          0
Programma       0
Lengte_sec      0
Kijkers         0
Temperatuur     0
Gevoelstemp     0
Regen           0
Sneeuw          0
Weercode        0
Bewolking       0
Windsnelheid    0
Zonnenschijn    0
dtype: int64


In [7]:
kijkcijfersWeer.dropna(inplace=True)
#csv aanmaken als het niet bestaat
csv_file_path = os.path.join('.','data csv','kijkcijfersWeerRaw.csv')
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
#data naar csv
kijkcijfersWeer.to_csv(csv_file_path, index=False)